In [2]:
import findspark

In [3]:
findspark.init('/home/samet/spark-3.2.1-bin-hadoop3.2')

In [4]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [8]:
from pyspark.ml.regression import LinearRegression

In [9]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

22/04/26 10:55:47 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [10]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [11]:
lr = LinearRegression(featuresCol='features', labelCol='label',
                      predictionCol='prediction')

In [21]:
lrModel = lr.fit(training)

22/04/26 11:04:14 WARN Instrumentation: [ae92b8f7] regParam is zero, which might cause numerical instability and overfitting.


In [13]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [14]:
lrModel.intercept

0.14228558260358093

In [15]:
training_summary = lrModel.summary

In [17]:
training_summary.rootMeanSquaredError

10.16309157133015

In [23]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

22/04/26 11:04:18 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [26]:
#split_object = all_data.randomSplit([0.7, 0.3])
train_data,test_data = all_data.randomSplit([0.7, 0.3])

In [30]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                340|
|   mean|0.43723945777014556|
| stddev| 10.740699285959804|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [31]:
test_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 161|
|   mean|-0.12397587405068466|
| stddev|   9.383170965665276|
|    min| -28.046018037776633|
|    max|  20.874246167942125|
+-------+--------------------+



In [32]:
correct_model = lr.fit(train_data)

22/04/26 11:08:00 WARN Instrumentation: [d4dbc82f] regParam is zero, which might cause numerical instability and overfitting.


In [33]:
test_result = correct_model.evaluate(test_data)

In [34]:
test_result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-27.005424029228394|
|-29.606640977922268|
| -22.73957643474098|
|  -23.0306096057989|
| -20.60035962899845|
| -17.70799136365038|
|-15.684606641429339|
| -14.51891888144272|
|-12.535544932836235|
|-18.347016610435436|
|-12.672138781388469|
|-13.426449756481412|
|  -11.2491502141405|
|-12.516908421228392|
|-16.778628292416798|
|-13.150576666078289|
|-15.508570942302503|
|-14.389405068268402|
|-12.472358078062973|
| -11.18021104458982|
+-------------------+
only showing top 20 rows



/home/samet/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [35]:
unlabeled_data = test_data.select('features')

In [36]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [37]:
predictions = correct_model.transform(unlabeled_data)

In [38]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -1.0405940085482406|
|(10,[0,1,2,3,4,5,...|  2.8011575494391945|
|(10,[0,1,2,3,4,5,...| -3.9966307478607463|
|(10,[0,1,2,3,4,5,...|  2.8185323468402292|
|(10,[0,1,2,3,4,5,...|  1.7544371560998686|
|(10,[0,1,2,3,4,5,...| -0.0956348250141379|
|(10,[0,1,2,3,4,5,...|  -1.034490192175751|
|(10,[0,1,2,3,4,5,...| -0.8569388418695768|
|(10,[0,1,2,3,4,5,...|  -2.813326222543019|
|(10,[0,1,2,3,4,5,...|  3.0122491305130943|
|(10,[0,1,2,3,4,5,...|  -2.090619471542657|
|(10,[0,1,2,3,4,5,...|0.005854980590655021|
|(10,[0,1,2,3,4,5,...| -1.9041853922250314|
|(10,[0,1,2,3,4,5,...|-0.46094030416371123|
|(10,[0,1,2,3,4,5,...|   3.856405189046378|
|(10,[0,1,2,3,4,5,...|  0.5920008772221005|
|(10,[0,1,2,3,4,5,...|  3.0171288647560908|
|(10,[0,1,2,3,4,5,...|   2.957602702727942|
|(10,[0,1,2,3,4,5,...|   1.143942141285191|
|(10,[0,1,2,3,4,5,...| 0.2342913